In [37]:
import requests
import pandas as pd
from urllib.request import urlretrieve
from zipfile import ZipFile 
import hashlib
import os
import shutil

import warnings
warnings.filterwarnings('ignore')

In [38]:
unique_df = pd.read_csv("games.csv")
unique_df

,id,versions,displayName,abbreviation
0,1,"[{'id': 68, 'md5': '811b027eaf99c2def7b933c520...",Super Mario Bros.,smb
1,2,"[{'id': 2372, 'md5': '71576D8339BD63198FCFC51A...",Super Mario Bros. 2 (U),smb2u
2,5,"[{'id': 146, 'md5': '097F3EB5211E106A5EA6EC43E...",Solomon's Key,solokey
3,6,"[{'id': 1162, 'md5': '50341185CAE666E292C39192...",Ghosts 'n Goblins,ghostsngoblins
4,7,"[{'id': 2254, 'md5': '52EB3F7E2C5FC765AA71F21C...",Castlevania,cv
...,...,...,...,...
917,4664,"[{'id': 5543, 'md5': 'd86872ed047e1cb8ba833642...",Shin Satomi Hakken-den: Hikari to Yami no Tatakai,sshhtynt
918,4667,"[{'id': 5546, 'md5': '505158927D6A4A03EC103EE0...",Arkanoid II,NaN
919,4669,"[{'id': 5550, 'md5': '632b28f125827ad670a0ddeb...",Blue Mario Bros. 3,NaN
920,4699,"[{'id': 5593, 'md5': 'd72fc0676ae28fbdde98fbbb...",Hokuto no Ken 4: Shichisei Haken Den — Hokuto ...,NaN


In [39]:
def idShaMap(versions, keyToFind):
   ids = []
   sha1s = []
   for version in versions.split("{")[1:]:
      dictEntries = version.split(",")
      for keyValPair in dictEntries:
         key = keyValPair[:keyValPair.find(":")].replace(" ", "").replace("'", "")
         value = keyValPair[keyValPair.find(":")+1:]
         if key != "name": value = value.replace(" ", "").replace("'", "")
         if key == "id": ids.append(int(value))
         if key == "sha1": sha1s.append(value)

   idShaDict = {}
   for i in range(len(ids)):
      idShaDict[ids[i]] = sha1s[i]
   return idShaDict

unique_df["sha1s"] = unique_df["versions"].map(lambda x: idShaMap(x, "id"))

In [40]:
def versionsStringExtract(versions, keyToFind):
   version1 = versions.split("{")[1:][0]
   dictEntries = version1.split(",")
   for keyValPair in dictEntries:
      key = keyValPair[:keyValPair.find(":")].replace(" ", "").replace("'", "")
      value = keyValPair[keyValPair.find(":")+1:]
      if key != "name": value = value.replace(" ", "").replace("'", "")
      if key == keyToFind: return value

# unique_df["md5"] = unique_df["versions"].map(lambda x: versionsStringExtract(x, "md5"))
# unique_df["sha1"] = unique_df["versions"].map(lambda x: versionsStringExtract(x, "sha1"))
unique_df["filename"] = unique_df["versions"].map(lambda x: versionsStringExtract(x, "name"))
unique_df["gameid"] = unique_df["versions"].map(lambda x: versionsStringExtract(x, "id"))
unique_df

,id,versions,displayName,abbreviation,sha1s,filename,gameid
0,1,"[{'id': 68, 'md5': '811b027eaf99c2def7b933c520...",Super Mario Bros.,smb,{68: 'ea343f4e445a9050d4b4fbac2c77d0693b1d0922...,'Super Mario Bros. (W) [!].nes',68
1,2,"[{'id': 2372, 'md5': '71576D8339BD63198FCFC51A...",Super Mario Bros. 2 (U),smb2u,{2372: '7DF0F595B074F587C6A1D8F47E031F045D540D...,'Super Mario Bros. 2 (U) (PRG0) [!]',2372
2,5,"[{'id': 146, 'md5': '097F3EB5211E106A5EA6EC43E...",Solomon's Key,solokey,{146: '18102689FD35C7D531A5E6241B06B748ACCAB2F...,"""Solomon's Key (U) [!]""",146
3,6,"[{'id': 1162, 'md5': '50341185CAE666E292C39192...",Ghosts 'n Goblins,ghostsngoblins,{1162: '22C99336AB8B9D3C1FA57E981766BC38CDB8FD...,'Makaimura (J) [!].nes',1162
4,7,"[{'id': 2254, 'md5': '52EB3F7E2C5FC765AA71F21C...",Castlevania,cv,{2254: '7A20C44F302FB2F1B7ADFFA6B619E3E1CAE7B5...,'Castlevania (U) (PRG1) [!].nes',2254
...,...,...,...,...,...,...,...
917,4664,"[{'id': 5543, 'md5': 'd86872ed047e1cb8ba833642...",Shin Satomi Hakken-den: Hikari to Yami no Tatakai,sshhtynt,{5543: 'ac0d556b93a6274b64f6f3beb3225100c84761...,'Shin Satomi Hakken-Den - Hikari to Yami no T...,5543
918,4667,"[{'id': 5546, 'md5': '505158927D6A4A03EC103EE0...",Arkanoid II,NaN,{5546: 'E352DDD87D911FEA5F637C907AAB744E11BB71...,'Arkanoid II (Japan) (En).nes',5546
919,4669,"[{'id': 5550, 'md5': '632b28f125827ad670a0ddeb...",Blue Mario Bros. 3,NaN,{5550: '9d7fc432091f7f32dfad961aff636632c77a2f...,'bluesmb3.nes',5550
920,4699,"[{'id': 5593, 'md5': 'd72fc0676ae28fbdde98fbbb...",Hokuto no Ken 4: Shichisei Haken Den — Hokuto ...,NaN,{5593: '8272ea411a97eb9c30a42a6d33fef50cd79403...,'Hokuto no Ken 4 - Shichisei Haken Den - Hoku...,5593


In [41]:
unique_df.iloc[0]["sha1s"].keys()

dict_keys([68, 280, 622, 639, 743, 2922, 4568, 4650, 4651, 4652, 4653, 4655, 4656, 4977])

In [42]:
noVersionsDF = unique_df.drop(columns="versions")
games_df = noVersionsDF
games_df

,id,displayName,abbreviation,sha1s,filename,gameid
0,1,Super Mario Bros.,smb,{68: 'ea343f4e445a9050d4b4fbac2c77d0693b1d0922...,'Super Mario Bros. (W) [!].nes',68
1,2,Super Mario Bros. 2 (U),smb2u,{2372: '7DF0F595B074F587C6A1D8F47E031F045D540D...,'Super Mario Bros. 2 (U) (PRG0) [!]',2372
2,5,Solomon's Key,solokey,{146: '18102689FD35C7D531A5E6241B06B748ACCAB2F...,"""Solomon's Key (U) [!]""",146
3,6,Ghosts 'n Goblins,ghostsngoblins,{1162: '22C99336AB8B9D3C1FA57E981766BC38CDB8FD...,'Makaimura (J) [!].nes',1162
4,7,Castlevania,cv,{2254: '7A20C44F302FB2F1B7ADFFA6B619E3E1CAE7B5...,'Castlevania (U) (PRG1) [!].nes',2254
...,...,...,...,...,...,...
917,4664,Shin Satomi Hakken-den: Hikari to Yami no Tatakai,sshhtynt,{5543: 'ac0d556b93a6274b64f6f3beb3225100c84761...,'Shin Satomi Hakken-Den - Hikari to Yami no T...,5543
918,4667,Arkanoid II,NaN,{5546: 'E352DDD87D911FEA5F637C907AAB744E11BB71...,'Arkanoid II (Japan) (En).nes',5546
919,4669,Blue Mario Bros. 3,NaN,{5550: '9d7fc432091f7f32dfad961aff636632c77a2f...,'bluesmb3.nes',5550
920,4699,Hokuto no Ken 4: Shichisei Haken Den — Hokuto ...,NaN,{5593: '8272ea411a97eb9c30a42a6d33fef50cd79403...,'Hokuto no Ken 4 - Shichisei Haken Den - Hoku...,5593


In [43]:
def compute_hash(filepath):
    hash = hashlib.sha1(open(filepath,'rb').read()).hexdigest()
    return hash

def find_file_by_hash(target_hash):
    root_dir = "."
    for subdir, _, files in os.walk(root_dir):
        for filename in files:
            filepath = os.path.join(subdir, filename)
            file_hash = compute_hash(filepath)
            if file_hash.lower() == target_hash.lower():
                print(f"Match found: {filepath}")
                return filepath
    print("No Matching ROM found.")
    return None

In [44]:
# entry = noVersionsDF.iloc[0]
# print(f"Searching for game {entry["displayName"]} (id: {entry["id"]}) with sha1: {entry["sha1"]}")
# find_file_by_hash(entry["sha1"])

In [45]:
pubs = pd.read_csv("pubs.csv")

In [46]:
import time

to_download = pubs.loc[pubs["id"]==1181]
# Download input replay
url = f"https://tasvideos.org/{1181}M?handler=Download"
filename = f"./{to_download["movieFileName"].item()}"
time.sleep(2)


In [47]:
for i in range(len(pubs)):
   row = pubs.iloc[i]
   print("\nLooking at run:", row["title"])
   gameId = row["gameId"]
   gameVersionId = row["gameVersionId"]
   gameName = games_df.loc[games_df["id"]==gameId]["displayName"].item()
   
   sha1Dict = games_df.loc[games_df["id"]==gameId]["sha1s"].item()
   if gameVersionId not in sha1Dict.keys():
      print("SHA1 not found for this game version ID.")
      continue
   sha1 = sha1Dict[gameVersionId]

   rom_path = find_file_by_hash(sha1)
   if rom_path is None: continue
   if not os.path.exists(f"./TAS_Videos/{gameName}"):
      os.makedirs(f"./TAS_Videos/{gameName}")

   # Download input replay
   url = f"https://tasvideos.org/{row["id"]}M?handler=Download"
   filename = f"./TAS_Videos/{gameName}/{row["movieFileName"]}"
   urlretrieve(url, filename + ".zip")
   with ZipFile(filename + ".zip", 'r') as zObject: 
      zObject.extractall(path=f"./TAS_Videos/{gameName}/")

   # Copy ROM into folder
   rom_name = rom_path[rom_path.rfind("/") + 1:]
   rom_destination = f"./TAS_Videos/{gameName}/{rom_name}"
   shutil.move(rom_path, rom_destination)

   # Save metadata
   row["SHA1"] = sha1
   row.to_csv(f"./TAS_Videos/{gameName}/run_metadata.csv")


Looking at run: NES Street Fighter 2010: The Final Fight by nesrocks in 17:39.98
Match found: ./TAS_Videos/Street Fighter 2010: The Final Fight/Street Fighter 2010 - The Final Fight (USA).nes

Looking at run: NES Deadly Towers (USA) by Luke in 15:20.27
No Matching ROM found.

Looking at run: NES Milon's Secret Castle by hisatoki in 05:28.43
No Matching ROM found.

Looking at run: NES Teenage Mutant Ninja Turtles II: The Arcade Game "1 player" by Baxter in 27:03.42
Match found: ./nintendo-entertainment-system-nes-roms-europeusa/3 Game Series Collections/Best-Of Lists/Top 10 Co-Op Games - CousinGaming.com/01 Teenage Mutant Ninja Turtles II - The Arcade Game (USA).nes

Looking at run: NES Dragon Fighter by nitsuja in 12:15.28
Match found: ./Extra/Dragon Fighter (USA).nes

Looking at run: NES The Legend of Kage by Bisqwit in 05:36.41
No Matching ROM found.

Looking at run: NES Totally Rad by Bag_of_Magic_Food in 12:07.57
Match found: ./Extra/Totally Rad (USA).nes

Looking at run: NES Whom

In [48]:
entry = pubs.loc[pubs["id"]==3728]
url = f"https://tasvideos.org/{entry["id"].item()}M?handler=Download"
filename = entry["movieFileName"].item()

urlretrieve(url, filename)

('happylee_mars608-smb-warpless.fm2', <http.client.HTTPMessage at 0x1368bc140>)

In [49]:
pubs.loc[pubs["id"]==3728].to_dict()

{'id': {350: 3728},
 'title': {350: 'NES Super Mario Bros. "warpless" by HappyLee & Mars608 in 18:36.78'},
 'branch': {350: 'warpless'},
 'goal': {350: 'warpless'},
 'gameGoalId': {350: 92},
 'emulatorVersion': {350: 'FCEUX 2.2.1'},
 'class': {350: 'Standard'},
 'systemCode': {350: 'NES'},
 'submissionId': {350: 5975},
 'gameId': {350: 1},
 'gameVersionId': {350: 68},
 'obsoletedById': {350: nan},
 'frames': {350: 67117},
 'rerecordCount': {350: 184530},
 'systemFrameRate': {350: 60.0988138974405},
 'movieFileName': {350: 'happylee_mars608-smb-warpless.fm2'},
 'additionalAuthors': {350: nan},
 'createTimestamp': {350: '2018-06-16T13:56:26'},
 'authors': {350: "['HappyLee', 'Mars608']"},
 'tags': {350: "['nowarps', 'bugs']"},
 'flags': {350: "['Verified', 'stars']"},
 'urls': {350: "['https://archive.org/download/supermariobros-tas-warpless-happylee_mars608/supermariobros-tas-warpless-happylee_mars608.mkv', 'https://archive.org/download/supermariobros-tas-warpless-happylee_mars608/super

In [50]:
gameDictList = []
for i in range(len(unique_df)):
   row = unique_df.iloc[i]
   versions = row["versions"].split("{")[1:]
   version1 = versions[0]
   dictEntries = version1.split(",")
   hashDict = {}
   for keyValPair in dictEntries:
      keyValPair = keyValPair.replace(" ", "").replace("'", "")
      colonLoc = keyValPair.find(":")
      key = keyValPair[:colonLoc]
      value = keyValPair[colonLoc+1:]
      if key in ["name", "md5", "sha1"]: hashDict[key] = value
   gameDictList.append(hashDict)
print(gameDictList)

[{'md5': '811b027eaf99c2def7b933c5208636de', 'sha1': 'ea343f4e445a9050d4b4fbac2c77d0693b1d0922', 'name': 'SuperMarioBros.(W)[!].nes'}, {'md5': '71576D8339BD63198FCFC51A92016D58', 'sha1': '7DF0F595B074F587C6A1D8F47E031F045D540DAE', 'name': 'SuperMarioBros.2(U)(PRG0)[!]'}, {'md5': '097F3EB5211E106A5EA6EC43E0C439DE', 'sha1': '18102689FD35C7D531A5E6241B06B748ACCAB2F6', 'name': '"SolomonsKey(U)[!]"'}, {'md5': '50341185CAE666E292C39192140A6338', 'sha1': '22C99336AB8B9D3C1FA57E981766BC38CDB8FD6E', 'name': 'Makaimura(J)[!].nes'}, {'md5': '52EB3F7E2C5FC765AA71F21C85F0770E', 'sha1': '7A20C44F302FB2F1B7ADFFA6B619E3E1CAE7B546', 'name': 'Castlevania(U)(PRG1)[!].nes'}, {'md5': '569678099D6D17C1DA174DAD04F78C71', 'sha1': 'DA48C842BC44DEF1C0EB2549941C6ADC65D6C379', 'name': 'DraculaII-NoroinoFuuin'}, {'md5': '8824842070A569E9BA7287BCDEA045A3', 'sha1': '5914D409EA027A96C2BB58F5136C5E7E9B2E8300', 'name': 'Rockman(Japan)(En).nes'}, {'md5': '055FB8DC626FB1FBADC0A193010A3E3F', 'sha1': 'FB51875D1FF4B0DEEE97E